In [29]:
import json
import time
import requests
from contextlib import nullcontext
from dataclasses import replace
from pickletools import float8
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from IPython.display import display
import openpyxl
from json import JSONDecoder
import schedule

In [30]:
def telegram_bot_sendtext(bot_message):
    
    bot_token = '6265667740:AAGu0tMfeMe09gD_YL0klKuNmH9OiQsVX9I'
    bot_chatID = '@P2P_magic'

    #send_text = f'https://api.telegram.org/bot{bot_token}/sendMessage?chat_id={bot_chatID}&parse_mode=Markdown&text={bot_message}'
    send_text = f'https://api.telegram.org/bot{bot_token}/sendMessage?chat_id={bot_chatID}&text={bot_message}'

    response = requests.get(send_text)

    return response.json()

#s = 'Привет!'
#telegram_bot_sendtext(s)

In [31]:
def extract_json_objects(text, decoder=JSONDecoder()):
    """Find JSON objects in text, and yield the decoded JSON data
    Does not attempt to look for JSON arrays, text, or other JSON types outside
    of a parent JSON object.
    """
    pos = 0
    while True:
        match = text.find('{', pos)
        if match == -1:
            break
        try:
            result, index = decoder.raw_decode(text[match:])
            yield result
            pos = match + index
        except ValueError:
            pos = match + 1

In [32]:
def job():

    deposit = 20000

    url1 = 'https://api.binance.com/api/v3/ticker/price?symbol=BTCRUB'
    item1 = requests.get(url1)
    BTCRUB = item1.json()['price']

    url2 = 'https://api.binance.com/api/v3/ticker/price?symbol=ETHRUB'
    item2 = requests.get(url2)
    ETHRUB = item2.json()['price']

    url3 = 'https://api.binance.com/api/v3/ticker/price?symbol=USDTRUB'
    item3 = requests.get(url3)
    USDTRUB = item3.json()['price']

    headers = {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
        "Cache-Control": "no-cache",
        "Connection": "keep-alive",
        "Content-Length": "123",
        "content-type": "application/json",
        "Host": "p2p.binance.com",
        "Origin": "https://p2p.binance.com",
        "Pragma": "no-cache",
        "TE": "Trailers",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0"
    }

    headers_bybit = {
        'Accept': 'application/json',
        'Accept-Language': 'ru-RU',
        'Accept-Encoding': 'gzip, deflate, br',
        'Host': 'api2.bybit.com',
        'Origin': 'https://www.bybit.com',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
        'Connection': 'keep-alive',
        'Referer': 'https://www.bybit.com/',
        'Content-Length': '114',
        }

    paytypes = [
        #'Advcash',
        #'RUBfiatbalance',
        'TinkoffNew',
        #'YandexMoneyNew',
        'QIWI',
        #'RosBankNew',
    ]

    for paytype in paytypes:

        coins = {1,2,3}
        method = int

        trades = {
            #'sell',
            'buy',
            }
        
        if paytype == 'TinkoffNew':
            method = 28
        if paytype == 'QIWI':
            method = 9
        
        results_huobi = []
        
        for coin in coins:
            for trade in trades:

                url = f'https://otc-api.trygofast.com/v1/data/trade-market?coinId={coin}&currency=11&tradeType={trade}&currPage=1&payMethod={method}&acceptOrder=-1&country=&blockType=general&online=1&range=0&amount={deposit}&isThumbsUp=false&isMerchant=false&isTraded=false&onlyTradable=false&isFollowed=false'
                page = requests.get(url)
                with open('result.json', 'w', encoding='utf-8') as f:
                    for result in extract_json_objects(page.text):
                        json.dump(result, f, ensure_ascii=False, indent=4)

                try:
                    [result['data'][0]["price"]]
                except:
                    results_huobi = results_huobi + [None]
                else:
                    if ([result['data'][0]["tradeMonthTimes"]][0] == 0 or [result['data'][0]["payTerm"]][0] > 15):
                        results_huobi = results_huobi + [result['data'][1]["price"]]
                    else:
                        results_huobi = results_huobi + [result['data'][0]["price"]]

        tradetypes = [
                    "SELL", 
                    #"BUY",
                    ]

        assets = [
            #'RUB',
            'BTC',
            'USDT',
            'ETH',
        ]

        results = []
        trademethod = []
        trade = []
        fiatUnit = []
        assetUnit = []
        results_bybit = []
        payment = []


        if paytype == 'TinkoffNew':
            payment = ["75"]
        if paytype == 'QIWI':
            payment = ["62"]

        for asset in assets:

            data_bybit = {
                "userId":"",
                "tokenId":asset,
                "currencyId":"RUB",
                "payment": payment,
                "side":"0", 
                "size":"10",
                "page":"1",
                "amount": str(deposit),
                }
            
            url = 'https://api2.bybit.com/fiat/otc/item/online'

            r = requests.post(url, headers=headers_bybit, json=data_bybit)
            dt_ = r.json()

            try:
                [dt_["result"]["items"][0]["price"]]
            except:
                results_bybit = results_bybit + [None]
            else:
                results_bybit = results_bybit + [dt_["result"]["items"][0]["price"]]


        for asset in assets:
            for tradetype in tradetypes:
                for i in range(1,2):
                    for j in range(1,2):

                        data = {
                            "asset": asset,
                            "fiat": "RUB",
                            #"merchantCheck": True,
                            "merchantCheck": False,
                            "page": i,
                            "payTypes": [paytype],
                            #"payTypes": [],
                            #"publisherType": 'merchant',
                            "publisherType": None,
                            "rows": j,
                            "tradeType": tradetype,
                            "transAmount":  str(deposit),
                        }

                        r = requests.post(
                            'https://p2p.binance.com/bapi/c2c/v2/friendly/c2c/adv/search', headers=headers, json=data)
                        dt = r.json()
                        #print((json.dumps(dt, indent=4)))
                        if [dt['data'][0]['adv']['price']] == None:
                            results = results + [None]
                            trademethod = trademethod + [None]
                            trade = trade + [None]
                            assetUnit = assetUnit + [None]
                            fiatUnit = fiatUnit + [None]
                        else:
                            results = results + [dt['data'][0]['adv']['price']]
                            trademethod = trademethod + [dt['data'][0]['adv']['tradeMethods'][0]['tradeMethodName']]
                            trade = trade + [dt['data'][0]['adv']['tradeType']]
                            assetUnit = assetUnit + [dt['data'][0]['adv']['asset']]
                            fiatUnit = fiatUnit + [dt['data'][0]['adv']['fiatUnit']]


        series = [
                #1, 
                BTCRUB,
                USDTRUB,
                ETHRUB, 
            ]

        dt_pd = pd.DataFrame({
            "trademethod": trademethod,
            "assetUnit": assetUnit,
            "trade": 'Вывод',
            "binance": results,
            "huobi": results_huobi,
            "bybit": results_bybit,
            "spot": series,   
        })

        dt_pd['binance'] = dt_pd['binance'].astype(float)
        dt_pd['huobi'] = dt_pd['huobi'].astype(float)
        dt_pd['bybit'] = dt_pd['bybit'].astype(float)
        dt_pd['spot'] = dt_pd['spot'].astype(float)
        dt_pd['percent_binance'] = (dt_pd['binance'] - dt_pd['spot']*1.012)/(dt_pd['spot']*1.012)*100
        dt_pd['percent_huobi'] = (dt_pd['huobi'] - dt_pd['spot']*1.012)/(dt_pd['spot']*1.012)*100
        dt_pd['percent_bybit'] = (dt_pd['bybit'] - dt_pd['spot']*1.012)/(dt_pd['spot']*1.012)*100
        dt_pd['profit_binance'] = np.round(dt_pd['percent_binance'] * deposit / 100,decimals=2)
        dt_pd['profit_huobi'] = np.round(dt_pd['percent_huobi'] * deposit / 100,decimals=2)
        dt_pd['profit_bybit'] = np.round(dt_pd['percent_bybit'] * deposit / 100,decimals=2)
        #dt_pd['good_price'] = np.round((dt_pd['binance'] * 1.0001), decimals=3)
        #dt_pd['good_loss'] = np.round((((1.012 * dt_pd['spot']) - (dt_pd['binance'] * 1.0001)) / (dt_pd['binance'] * 1.0001) * 100),decimals=2)

        def highlight_percent(val):
            color = 'yellowgreen' if val > 2.0 else ' '
            return 'background-color: %s' % color

        def highlight_loss(val):
            color = '#6ac6d0' if val > -1.0 else ' '
            return 'background-color: %s' % color
            '''
        display(dt_pd.style
            .highlight_max(color='yellowgreen', subset=['percent_binance','percent_huobi','percent_bybit'],axis=None)
            .highlight_max(color='#6ac6d0', subset=['profit_binance','profit_huobi','profit_bybit'],axis=None)
            .format(precision=2, na_rep='', thousands=" ")
            .applymap(highlight_percent, subset=['percent_binance'])
            .applymap(highlight_percent, subset=['percent_huobi'])
            .applymap(highlight_percent, subset=['percent_bybit'])
            #.applymap(highlight_loss, subset=['good_loss'])
            )
            '''

        status_0 = dt_pd[['percent_binance','percent_huobi','percent_bybit']].max().max()
        status = f"Максимальный процент  = {np.round(status_0,decimals=2)}%"
        if status_0 > 1.5:
            telegram_bot_sendtext(status)
            time.sleep(5)

job()

#schedule.every().hour.at(":01").do(job)
schedule.every(20).minutes.do(job)

while True:
    schedule.run_pending()
    time.sleep(1)
    

,trademethod,assetUnit,trade,binance,huobi,bybit,spot,percent_binance,percent_huobi,percent_bybit,profit_binance,profit_huobi,profit_bybit
0,Tinkoff,BTC,Вывод,1 746 522.01,1 706 336.08,1 759 602.33,1 740 792.00,-0.86,-3.14,-0.12,-172.10,-628.32,-23.60
1,Tinkoff,USDT,Вывод,78.05,78.15,78.24,77.06,0.08,0.21,0.33,16.74,42.39,65.47
2,Tinkoff,ETH,Вывод,122 523.28,120 200.00,123 012.47,121 964.60,-0.73,-2.62,-0.34,-146.63,-523.09,-67.36


,trademethod,assetUnit,trade,binance,huobi,bybit,spot,percent_binance,percent_huobi,percent_bybit,profit_binance,profit_huobi,profit_bybit
0,QIWI,BTC,Вывод,1 725 952.78,1 706 412.26,1 701 124.12,1 740 792.00,-2.03,-3.14,-3.44,-405.62,-627.46,-687.50
1,MTS-Bank,USDT,Вывод,77.20,78.03,77.96,77.06,-1.01,0.06,-0.03,-201.25,11.61,-6.34
2,MTS-Bank,ETH,Вывод,122 085.01,117 756.84,116 302.58,121 964.60,-1.09,-4.59,-5.77,-217.64,-918.97,-1 154.61


,trademethod,assetUnit,trade,binance,huobi,bybit,spot,percent_binance,percent_huobi,percent_bybit,profit_binance,profit_huobi,profit_bybit
0,Tinkoff,BTC,Вывод,1 735 946.20,1 699 687.01,1 737 000.00,1 726 419.00,-0.64,-2.72,-0.58,-128.09,-543.16,-116.03
1,Tinkoff,USDT,Вывод,78.06,78.14,78.28,77.14,-0.01,0.10,0.27,-1.46,19.04,54.91
2,MTS-Bank,ETH,Вывод,121 010.42,120 200.00,118 830.34,120 494.40,-0.76,-1.43,-2.55,-152.52,-285.44,-510.08


,trademethod,assetUnit,trade,binance,huobi,bybit,spot,percent_binance,percent_huobi,percent_bybit,profit_binance,profit_huobi,profit_bybit
0,Юmoney,BTC,Вывод,1 710 683.63,1 692 949.58,1 683 955.52,1 726 419.00,-2.09,-3.10,-3.62,-417.28,-620.29,-723.25
1,Raiffeisenbank,USDT,Вывод,77.21,78.03,77.96,77.14,-1.10,-0.05,-0.14,-219.22,-9.14,-27.07
2,MTS-Bank,ETH,Вывод,121 010.42,116 857.04,115 136.41,120 494.40,-0.76,-4.17,-5.58,-152.52,-833.73,-1 115.94


,trademethod,assetUnit,trade,binance,huobi,bybit,spot,percent_binance,percent_huobi,percent_bybit,profit_binance,profit_huobi,profit_bybit
0,Tinkoff,BTC,Вывод,1 744 632.03,1 722 000.00,1 737 000.00,1 726 552.00,-0.15,-1.45,-0.59,-30.20,-289.26,-117.56
1,Tinkoff,USDT,Вывод,78.24,78.05,78.27,77.18,0.17,-0.07,0.21,34.27,-14.38,41.95
2,Tinkoff,ETH,Вывод,122 078.21,120 200.00,120 332.64,120 494.40,0.11,-1.43,-1.32,22.61,-285.44,-263.69


...

,trademethod,assetUnit,trade,binance,huobi,bybit,spot,percent_binance,percent_huobi,percent_bybit,profit_binance,profit_huobi,profit_bybit
0,Tinkoff,BTC,Вывод,1 753 066.92,1 728 000.00,1 745 000.00,1 730 338.00,0.11,-1.32,-0.35,22.44,-263.86,-69.69
1,Tinkoff,USDT,Вывод,78.27,78.20,78.27,77.17,0.22,0.13,0.22,44.55,26.62,44.55
2,Tinkoff,ETH,Вывод,122 807.85,120 200.00,121 800.00,121 213.40,0.11,-2.01,-0.71,22.81,-402.38,-141.51


,trademethod,assetUnit,trade,binance,huobi,bybit,spot,percent_binance,percent_huobi,percent_bybit,profit_binance,profit_huobi,profit_bybit
0,QIWI,BTC,Вывод,1 723 098.85,1 696 660.19,1 690 000.00,1 730 338.00,-1.60,-3.11,-3.49,-319.84,-621.80,-697.87
1,QIWI,USDT,Вывод,77.54,78.04,78.00,77.17,-0.71,-0.07,-0.12,-142.40,-14.35,-24.60
2,Юmoney,ETH,Вывод,120 663.93,118 876.10,115 393.60,121 213.40,-1.63,-3.09,-5.93,-326.74,-618.23,-1 186.02


,trademethod,assetUnit,trade,binance,huobi,bybit,spot,percent_binance,percent_huobi,percent_bybit,profit_binance,profit_huobi,profit_bybit
0,Tinkoff,BTC,Вывод,1 750 470.09,1 727 068.76,1 745 001.00,1 730 000.00,-0.02,-1.35,-0.33,-3.31,-270.64,-65.79
1,Tinkoff,USDT,Вывод,78.19,78.34,78.33,77.17,0.12,0.31,0.30,24.06,62.48,59.92
2,Tinkoff,ETH,Вывод,122 669.01,120 200.00,121 900.00,121 222.60,-0.01,-2.02,-0.63,-1.35,-403.87,-126.72


,trademethod,assetUnit,trade,binance,huobi,bybit,spot,percent_binance,percent_huobi,percent_bybit,profit_binance,profit_huobi,profit_bybit
0,QIWI,BTC,Вывод,1 724 289.01,1 696 419.44,1 690 000.00,1 730 000.00,-1.51,-3.10,-3.47,-302.39,-620.77,-694.10
1,QIWI,USDT,Вывод,77.53,78.08,78.05,77.17,-0.72,-0.02,-0.06,-144.96,-4.11,-11.79
2,QIWI,ETH,Вывод,120 872.78,118 909.80,115 471.50,121 222.60,-1.47,-3.07,-5.87,-294.19,-614.21,-1 174.75


KeyboardInterrupt: 